Implementation is from Book **Hands on Large Language Models** by Jay Alammar - Chapter 4.

#### **Error:**
```
Invalid pattern: '**' can only be an entire path component
```
**Solution:**
This most probably was caused due to runtime had older version of libraries
- datasets
- huggingface_hub
- fsspecs

So installed those and error resolved.

In [ ]:
!pip install -U datasets huggingface_hub fsspec

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 11.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; pla

# Data Preparation

### About dataset

This is twitter sentiment dataset with a positive(0) and negative(1) review in each example.

It already is splitted in train, val, test so you can access ith with key like `data['train']`

In [ ]:
from datasets import load_dataset

dataset = load_dataset("rotten_tomatoes")
print(dataset)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

train.parquet:   0%|          | 0.00/699k [00:00<?, ?B/s]

validation.parquet:   0%|          | 0.00/90.0k [00:00<?, ?B/s]

test.parquet:   0%|          | 0.00/92.2k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/8530 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1066 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1066 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 8530
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 1066
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 1066
    })
})


In [ ]:
dataset['train'][0]

{'text': 'the rock is destined to be the 21st century\'s new " conan " and that he\'s going to make a splash even greater than arnold schwarzenegger , jean-claud van damme or steven segal .',
 'label': 1}

In [ ]:
dataset['train'][0, -1, 2]

{'text': ['the rock is destined to be the 21st century\'s new " conan " and that he\'s going to make a splash even greater than arnold schwarzenegger , jean-claud van damme or steven segal .',
  'things really get weird , though not particularly scary : the movie is all portent and no content .',
  'effective but too-tepid biopic'],
 'label': [1, 0, 1]}

# Classification

We classify using the `Twitter-RoBERTa-base for Sentiment Analysis` model.

- Freze the model and only get the embeddings

In [ ]:
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
from transformers import pipeline

# Path to our HF model
model_path = "cardiffnlp/twitter-roberta-base-sentiment-latest"
# Load model into pipeline
pipe = pipeline(
 model=model_path,
 tokenizer=model_path,
 return_all_scores=True,
 device=device
)

config.json:   0%|          | 0.00/929 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/501M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/501M [00:00<?, ?B/s]

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

Device set to use cpu
/usr/local/lib/python3.11/dist-packages/transformers/pipelines/text_classification.py:106: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


#### Run on test set

In [ ]:
import numpy as np
from tqdm import tqdm
from transformers.pipelines.pt_utils import KeyDataset

# Run inference
y_pred = []
for output in tqdm(pipe(KeyDataset(dataset["test"], "text")),
                                    total=len(dataset["test"])):
  negative_score = output[0]["score"]
  positive_score = output[2]["score"]
  assignment = np.argmax([negative_score, positive_score])
  y_pred.append(assignment)


100%|██████████| 1066/1066 [04:24<00:00,  4.03it/s]


In [ ]:
from sklearn.metrics import classification_report

def evaluate_performance(y_true, y_pred):
 """Create and print the classification report"""
 performance = classification_report(
 y_true, y_pred,
 target_names=["Negative Review", "Positive Review"]
 )
 print(performance)

In [ ]:
evaluate_performance(dataset["test"]["label"], y_pred)

                 precision    recall  f1-score   support

Negative Review       0.76      0.88      0.81       533
Positive Review       0.86      0.72      0.78       533

       accuracy                           0.80      1066
      macro avg       0.81      0.80      0.80      1066
   weighted avg       0.81      0.80      0.80      1066



In [ ]:
dataset['test'][100],y_pred[100]

({'text': 'symbolically , warm water under a red bridge is a celebration of feminine energy , a tribute to the power of women to heal .',
  'label': 1},
 np.int64(1))

# general-purpose embedding models

General-purpose embedding models are models that convert raw input data (like text, images, code, etc.) into dense vector representations — embeddings — that can be used across many downstream tasks, without task-specific fine-tuning.

<img src="../images/experiments/text_classification_llm_pretrained/pretrained_embeddings_classification.png"  alt="Pretrained Embeddings"/>

A major benefit of this separation is that we do not need to fine-tune our
embedding model, which can be costly. In contrast, we can train a classifier,like a logistic regression, on the CPU instead

In [ ]:
from sentence_transformers import SentenceTransformer

# Load model
model = SentenceTransformer("sentence-transformers/all-mpnet-base-v2")

# Convert text to embeddings
train_embeddings = model.encode(dataset["train"]["text"],
                                show_progress_bar=True)
test_embeddings = model.encode(dataset["test"]["text"],
                               show_progress_bar=True)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/267 [00:00<?, ?it/s]

Batches:   0%|          | 0/34 [00:00<?, ?it/s]

In [ ]:
train_embeddings.shape

(8530, 768)

In [ ]:
from sklearn.linear_model import LogisticRegression

# Train a logistic regression on our train embeddings
clf = LogisticRegression(random_state=42)
clf.fit(train_embeddings, dataset["train"]["label"])

LogisticRegression(random_state=42)

In [ ]:
# Predict previously unseen instances
y_pred = clf.predict(test_embeddings)
evaluate_performance(dataset["test"]["label"], y_pred)

                 precision    recall  f1-score   support

Negative Review       0.85      0.86      0.85       533
Positive Review       0.86      0.85      0.85       533

       accuracy                           0.85      1066
      macro avg       0.85      0.85      0.85      1066
   weighted avg       0.85      0.85      0.85      1066



In [ ]:
dataset['test'][100],y_pred[100]

({'text': 'symbolically , warm water under a red bridge is a celebration of feminine energy , a tribute to the power of women to heal .',
  'label': 1},
 np.int64(1))

# Zero Shot Classification

Predicts labels without even being trained on data

In [ ]:
# Create embeddings for our labels
label_embeddings = model.encode(["A negative review", "A positive review"])

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
# Find the best matching label for each document
sim_matrix = cosine_similarity(test_embeddings, label_embeddings)
y_pred = np.argmax(sim_matrix, axis=1)

NameError: name 'label_embeddings' is not defined

In [ ]:
evaluate_performance(dataset["test"]["label"], y_pred)

# Generative Models

## Text-to-Text Transfer Transformer

In [ ]:
# Load our model
pipe = pipeline(
 "text2text-generation",
 model="google/flan-t5-small",
 device=device
)

In [ ]:
# Prepare our data
prompt = "Is the following sentence positive or negative? "
data = dataset.map(lambda example: {"t5": prompt + example['text']})
data

In [ ]:
# Run inference
y_pred = []
for output in tqdm(pipe(KeyDataset(data["test"], "t5")),
total=len(data["test"])):
 text = output[0]["generated_text"]
 y_pred.append(0 if text == "negative" else 1)

In [ ]:
evaluate_performance(data["test"]["label"], y_pred)

## Chat GPT for Classification

In [ ]:
import openai
# Create client
client = openai.OpenAI(api_key="")

In [ ]:
def chatgpt_generation(prompt, document, model="gpt-3.5-turbo-0125"):
 """Generate an output based on a prompt and an input
document."""

 messages=[
 {
 "role": "system",
 "content": "You are a helpful assistant."
 },
 {
 "role": "user",
 "content": prompt.replace("[DOCUMENT]", document)
 }
 ]
 chat_completion = client.chat.completions.create(
 messages=messages,
 model=model,
 temperature=0
 )
 return chat_completion.choices[0].message.content

In [ ]:
# Define a prompt template as a base
prompt = """Predict whether the following document is a positive
or negative movie review:
[DOCUMENT]
If it is positive return 1 and if it is negative return 0. Do not
give any other answers.
"""
# Predict the target using GPT
document = "unpretentious , charming , quirky , original"
chatgpt_generation(prompt, document)

In [ ]:
# You can skip this if you want to save your (free) credits
predictions = [
 chatgpt_generation(prompt, doc) for doc in tqdm(data["test"]
["text"])
]

In [ ]:
# Extract predictions
y_pred = [int(pred) for pred in predictions]

In [ ]:
# Evaluate performance
evaluate_performance(data["test"]["label"], y_pred)